In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from imblearn.over_sampling import  SMOTE
from imblearn.under_sampling import RandomUnderSampler
import tensorflow as tf

In [ ]:
# Selecting cleaned data from your local drive
from google.colab import files
uploaded = files.upload()

Saving post_clean.csv to post_clean.csv


In [ ]:
# reading in csv to pandas dataframe
import io
df_cleaned = pd.read_csv(io.BytesIO(uploaded['post_clean.csv']))

In [ ]:
# checking the distribution of our Target variable
df_cleaned['stroke'].value_counts(normalize=True)

0    0.951262
1    0.048738
Name: stroke, dtype: float64

In [ ]:
# Creating dummies for categorical variables
df_new= pd.get_dummies(df_cleaned)
df_new.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never,work_type_Private,work_type_Self-employed,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.60,1,0,1,0,1,0,0,1,0,0,1,0,1,0,0
1,61.0,0,0,202.21,28.89,1,1,0,0,1,0,0,0,1,1,0,0,0,1,0
2,80.0,0,1,105.92,32.50,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0
3,49.0,0,0,171.23,34.40,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1
4,79.0,1,0,174.12,24.00,1,1,0,0,1,0,0,0,1,1,0,0,0,1,0


In [ ]:
# creating our Feautres and Target 
y= df_new['stroke']
X= df_new.drop(columns="stroke")

In [ ]:
# splitting for training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=1)

In [ ]:
# Create an StandarScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled= X_scaler.transform(X_test)

In [ ]:
pd.Series(y_train).value_counts(normalize=True)

0    0.951188
1    0.048812
Name: stroke, dtype: float64

## **Original Model**

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_dimen= len(X_train_scaled[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units= 10,activation='relu',input_dim= input_dimen))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units= 8,activation='leaky_relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units= 1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                200       
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 297
Trainable params: 297
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model
fit_model_1= nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
120/120 [==============================] - 4s 3ms/step - loss: 0.4773 - accuracy: 0.7820
Epoch 2/100
120/120 [==============================] - 0s 3ms/step - loss: 0.2221 - accuracy: 0.9509
Epoch 3/100
120/120 [==============================] - 0s 2ms/step - loss: 0.1805 - accuracy: 0.9512
Epoch 4/100
120/120 [==============================] - 0s 2ms/step - loss: 0.1722 - accuracy: 0.9512
Epoch 5/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1677 - accuracy: 0.9512
Epoch 6/100
120/120 [==============================] - 0s 4ms/step - loss: 0.1647 - accuracy: 0.9512
Epoch 7/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1621 - accuracy: 0.9512
Epoch 8/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1599 - accuracy: 0.9512
Epoch 9/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1582 - accuracy: 0.9512
Epoch 10/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1568 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

40/40 - 0s - loss: 0.1792 - accuracy: 0.9507 - 206ms/epoch - 5ms/step
Loss: 0.17916595935821533, Accuracy: 0.9507042169570923


In [ ]:
# checking ROC Score
roc_1 = roc_auc_score(y_test,nn.predict(X_test_scaled))
print(roc_1)

40/40 [==============================] - 0s 1ms/step
0.7910587330220713


In [ ]:
# Confusion Matrix
index= ["Actual No Stroke", "Actual Stroke"]
columns= ["Predicted No Stroke","Predicted Stroke"]
predictions = nn.predict(X_test_scaled)
threshold = (predictions>0.5).astype(int)
cm_1= confusion_matrix(y_test, threshold)
pd.DataFrame(cm_1,index=index,columns=columns)

40/40 [==============================] - 0s 1ms/step


,Predicted No Stroke,Predicted Stroke
Actual No Stroke,1215,1
Actual Stroke,62,0


In [ ]:
# Classification Matrix
print(classification_report(y_test,threshold))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1216
           1       0.00      0.00      0.00        62

    accuracy                           0.95      1278
   macro avg       0.48      0.50      0.49      1278
weighted avg       0.91      0.95      0.93      1278



## **SMOTE MODEL**

In [ ]:
# Creating Random Over Sampler to balance dataset
smote= SMOTE(sampling_strategy=.2,random_state=9)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [ ]:
# Checking distribution of Target Variable
pd.Series(y_train_resampled).value_counts(normalize=True)

0    0.833486
1    0.166514
Name: stroke, dtype: float64

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train_resampled)

# Scale the data
X_train_r_scaled = X_scaler.transform(X_train_resampled)


In [ ]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimen= len(X_train_r_scaled[0])
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units= 10,activation='relu',input_dim= input_dimen))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units= 8,activation='leaky_relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units= 1,activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                200       
                                                                 
 dense_4 (Dense)             (None, 8)                 88        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 297
Trainable params: 297
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn2.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
# Train the model
fit_model= nn2.fit(X_train_r_scaled,y_train_resampled,epochs=100)

Epoch 1/100
137/137 [==============================] - 1s 2ms/step - loss: 0.4762 - accuracy: 0.8259
Epoch 2/100
137/137 [==============================] - 0s 2ms/step - loss: 0.3532 - accuracy: 0.8335
Epoch 3/100
137/137 [==============================] - 0s 2ms/step - loss: 0.2985 - accuracy: 0.8449
Epoch 4/100
137/137 [==============================] - 0s 2ms/step - loss: 0.2486 - accuracy: 0.8971
Epoch 5/100
137/137 [==============================] - 0s 2ms/step - loss: 0.2036 - accuracy: 0.9318
Epoch 6/100
137/137 [==============================] - 0s 2ms/step - loss: 0.1780 - accuracy: 0.9405
Epoch 7/100
137/137 [==============================] - 0s 2ms/step - loss: 0.1642 - accuracy: 0.9460
Epoch 8/100
137/137 [==============================] - 0s 2ms/step - loss: 0.1578 - accuracy: 0.9483
Epoch 9/100
137/137 [==============================] - 0s 2ms/step - loss: 0.1527 - accuracy: 0.9501
Epoch 10/100
137/137 [==============================] - 0s 2ms/step - loss: 0.1501 - accura

In [ ]:
# Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

40/40 - 0s - loss: 0.3040 - accuracy: 0.8670 - 236ms/epoch - 6ms/step
Loss: 0.30399197340011597, Accuracy: 0.8669796586036682


In [ ]:
# Checking ROC Score
roc_2 = roc_auc_score(y_test,nn2.predict(X_test_scaled))
print(roc_2)

40/40 [==============================] - 0s 2ms/step
0.8114654074702886


In [ ]:
# Confusion Matrix
predictions_2 = nn2.predict(X_test_scaled)
threshold = (predictions_2>0.5).astype(int)
cm_2= confusion_matrix(y_test, threshold)
pd.DataFrame(cm_2,index=index,columns=columns)

40/40 [==============================] - 0s 2ms/step


,Predicted No Stroke,Predicted Stroke
Actual No Stroke,1078,138
Actual Stroke,32,30


In [ ]:
# Classification Matrix
print(classification_report(y_test,threshold))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93      1216
           1       0.18      0.48      0.26        62

    accuracy                           0.87      1278
   macro avg       0.57      0.69      0.59      1278
weighted avg       0.93      0.87      0.89      1278



## **Under Sampling Model**

In [ ]:
rus=RandomUnderSampler(sampling_strategy=.15,random_state=2)
X_train_resampled_2, y_train_resampled_2 = rus.fit_resample(X_train, y_train)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train_resampled_2)

# Scale the data
X_train_r2_scaled = X_scaler.transform(X_train_resampled_2)


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_dimen= len(X_train_r2_scaled[0])
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units= 10,activation='relu',input_dim= input_dimen))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units= 8,activation='leaky_relu'))

# Output layer
nn3.add(tf.keras.layers.Dense(units= 1,activation='sigmoid'))

# Check the structure of the model
nn3.summary()

# Compile the model
nn3.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                200       
                                                                 
 dense_7 (Dense)             (None, 8)                 88        
                                                                 
 dense_8 (Dense)             (None, 1)                 9         
                                                                 
Total params: 297
Trainable params: 297
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model
fit_model_3= nn3.fit(X_train_r2_scaled,y_train_resampled_2,epochs=100)

Epoch 1/100
45/45 [==============================] - 1s 3ms/step - loss: 0.9266 - accuracy: 0.3210
Epoch 2/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6371 - accuracy: 0.6218
Epoch 3/100
45/45 [==============================] - 0s 2ms/step - loss: 0.5071 - accuracy: 0.8144
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 0.4345 - accuracy: 0.8521
Epoch 5/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3942 - accuracy: 0.8653
Epoch 6/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3693 - accuracy: 0.8681
Epoch 7/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3529 - accuracy: 0.8674
Epoch 8/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3420 - accuracy: 0.8709
Epoch 9/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3347 - accuracy: 0.8709
Epoch 10/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3295 - accuracy: 0.8716
Epoch 11/

In [ ]:
# Evaluate the model using the test data
model_loss_3, model_accuracy_3 = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

40/40 - 0s - loss: 0.2202 - accuracy: 0.9210 - 166ms/epoch - 4ms/step
Loss: 0.22020065784454346, Accuracy: 0.9209702610969543


In [ ]:
# Checking ROC Score
roc_3 = roc_auc_score(y_test,nn3.predict(X_test_scaled))
print(roc_3)

40/40 [==============================] - 0s 1ms/step
0.8140784168081494


In [ ]:
# Confusion Matrix
predictions_3 = nn3.predict(X_test_scaled)
threshold = (predictions_3>0.5).astype(int)
cm_3= confusion_matrix(y_test, threshold)
pd.DataFrame(cm_3,index=index,columns=columns)

40/40 [==============================] - 0s 1ms/step


,Predicted No Stroke,Predicted Stroke
Actual No Stroke,1157,59
Actual Stroke,42,20


In [ ]:
# Classification Matrix
print(classification_report(y_test,threshold))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96      1216
           1       0.25      0.32      0.28        62

    accuracy                           0.92      1278
   macro avg       0.61      0.64      0.62      1278
weighted avg       0.93      0.92      0.93      1278



In [ ]:
model1_dict = {"roc_score":[roc_1,roc_2,roc_3],"model_accuracy":[model_accuracy,model_accuracy_2,model_accuracy_3],'Model_Type':index}
index=['Original_base','Smote','Random_Under_Sample']

In [ ]:
score_df= pd.DataFrame(model1_dict)
score_df.set_index("Model_Type")

,roc_score,model_accuracy
Model_Type,,
Original_base,0.791059,0.950704
Smote,0.811465,0.866980
Random_Under_Sample,0.814078,0.920970
